In [ ]:
from src.model.segmentation_model import CSegmentationModel
from src.utils.parameters import CParameters

# Debugging
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
model = CSegmentationModel(CParameters())
model.loadDatasets()
model.createDataLoaders()

In [ ]:
print(model.m_trainingDataset.__len__())
print(model.m_validationDataset.__len__())

In [ ]:
# Debugging
img, mask = model.m_validationDataset.__getitem__(4)

fig, axs = plt.subplots(1, 2)
axs[0].imshow(np.transpose(img.numpy(), (1, 2, 0)))
axs[1].imshow(mask.numpy())

In [ ]:
model.train()